## EfficientNetB0
### Database: [Plant Diseases Training Dataset](https://www.kaggle.com/datasets/nirmalsankalana/plant-diseases-training-dataset/data)

In [ ]:
#Import nessesary packages, libraries and global variables

import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from  PIL import Image
import numpy as np
from tensorflow.keras import Input, Model
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.utils import split_dataset
import tensorflow as tf
from tensorflow import keras

import sys
sys.path.append('../modeling')

RSEED = 42
dataset_path = '../data/train/'

In [ ]:
def load_preprocess_split_train_val(data_path):
    ''' 
    Function needs filefath as parameter, it will create a validation dataset of 20% of the total df, 
    Needs an RSEED as global variable,
    Image will be cropped to 1:1 and altered to 224 x 224
    '''
    image = tf.keras.utils.image_dataset_from_directory(
        data_path, 
        validation_split = 0.2,
        subset = "both", 
        seed = RSEED,
        image_size = (224, 224),
        crop_to_aspect_ratio = True,
        label_mode = 'categorical'
    )
    return image 

In [ ]:
train_ds, val_ds = load_preprocess_split_train_val(dataset_path)

In [ ]:
#check an example 
val_ds.take(1)

------------------------------------------------------------------------------------------------------------------------------------------
#### Developing a model
------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
import tensorflow.keras.backend as K
K.clear_session()

In [ ]:
#define model

model = keras.applications.EfficientNetB0(
    include_top=False
)

In [ ]:
#get overview of the model architecture

model.summary()

In [ ]:
from keras import layers
IMG_SIZE = 224
BATCH_SIZE = 32

def build_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3)) # Define the input layer with the shape of input images
    model = EfficientNetB0(include_top=False, input_tensor=inputs, weights="imagenet") # Load the EfficientNetB0 model pretrained on ImageNet without the top classification layer
                                                                                        # Use the input layer defined above

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output) # Apply global average pooling to the output of the base layers
    x = layers.BatchNormalization()(x) # Apply batch normalization to normalize the activations of the previous layer


    top_dropout_rate = 0.2 # Define the dropout rate
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x) # Apply dropout regularization to the previous layer
    outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x) # Add a dense layer for classification with softmax activation

    # Compile
    model = keras.Model(inputs, outputs, name="EfficientNet") # Construct the final model with the specified input and output layers
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-2) # Define the optimizer with a learning rate of 0.01 using the Adam optimizer
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    
    return model

In [ ]:
model = build_model(num_classes=39)

epochs = 12  # @param {type: "slider", min:8, max:80}
#hist = model.fit(train_ds, epochs=epochs, validation_data=val_ds)


In [ ]:


def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

In [ ]:
plot_hist(hist)

#### next we want to unfreeze a couple of layers and retrain with own data

In [ ]:
#next we want to unfreeze 10 layers and retrain 

import tensorflow as tf

def unfreeze_model_and_clone(model):
    # Clone the original model
    unfrozen_model = tf.keras.models.clone_model(model)
    unfrozen_model.set_weights(model.get_weights())  # Copy weights

    # Unfreeze the top 10 layers while leaving BatchNorm layers frozen
    for layer in unfrozen_model.layers[-10:]:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-5)
    unfrozen_model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    
    return unfrozen_model


In [ ]:
# Create a new model with unfrozen layers
unfrozen_model2 = unfreeze_model_and_clone(model)

epochs = 8
hist = unfrozen_model2.fit(train_ds, epochs=epochs, validation_data=val_ds)

plot_hist(hist)


In [ ]:
# Create a new model with unfrozen layers
unfrozen_model3 = unfreeze_model_and_clone(model)

epochs = 12
hist3 = unfrozen_model3.fit(train_ds, epochs=epochs, validation_data=val_ds)

# Save the model to disk
unfrozen_model3.save("unfrozen_model3.h5")

plot_hist(hist3)

In [ ]:
# Plot training & validation loss values
plt.plot(hist3.history['loss'])
plt.plot(hist3.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Save the trained model to the specified directory
model_dir = "../models/"
model_filename = "efficient_unfrozen_12.h5"
unfrozen_model3.save(os.path.join(model_dir, model_filename))

# You can load it back with keras.models.load_model().

------------------------------------------------------------------------------------------------------------------------------------------
#### Testing of the model with unseen data
------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:


def load_test(data_path):
    ''' 
    Function needs filepath as parameter, it will create a validation dataset of 20% of the total df, 
    Needs an RSEED as global variable,
    Image will be cropped to 1:1 and altered to 224 x 224
    '''
    image_dataset = tf.keras.utils.image_dataset_from_directory(
        data_path,
        image_size = (224, 224),
        crop_to_aspect_ratio = True,
        label_mode = 'categorical',
        shuffle = False
    )
    return image_dataset 

In [ ]:
dataset_test_path = '../data/test/'

test_ds = load_test(dataset_test_path)

In [ ]:
#check an example 
test_ds.take(1)

In [ ]:
# Make predictions on the test dataset
predictions = unfrozen_model3.predict(test_ds)

------------------------------------------------------------------------------------------------------------------------------------------
#### Plotting the results and getting evaluation metrics
------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Step 1: Get the true labels from the test dataset
y_true = []
for filepath in test_ds.file_paths:
    label = os.path.basename(os.path.dirname(filepath))
    y_true.append(label)

# Extract unique class labels from your training data
classes = sorted(set(y_true))

# Step 2: Convert true labels to indices using the same mapping used during training
class_to_index = {cls: i for i, cls in enumerate(classes)}
y_true_indices = np.array([class_to_index[label] for label in y_true])

# Step 3: Use your model to make predictions on the test dataset
y_pred_probabilities = unfrozen_model3.predict(test_ds)

# Step 4: Convert the predicted class probabilities to class labels
y_pred_indices = np.argmax(y_pred_probabilities, axis=1)
y_pred = [classes[i] for i in y_pred_indices]

# Step 5: Generate the classification report
report = classification_report(y_true, y_pred)
print("Classification Report:")
print(report)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

cm = confusion_matrix(y_true, y_pred)

# Display the confusion matrix using seaborn heatmap with green color palette
plt.figure(figsize=(14, 14))
sns.heatmap(cm, annot=False, cmap="Greens", xticklabels=classes, yticklabels=classes)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

#### Next we should try with the suffle parameter turned on (while loarding)

-----------------------------------------------------------------------------------------------------------------------------------------------
#### Retrain with augmented Data
-----------------------------------------------------------------------------------------------------------------------------------------------

#### with augmented training Data

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define your data augmentation parameters
# Define your data augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.8, 1.2],  # Adjust brightness by random factor between 0.8 and 1.2
    horizontal_flip=True,
    fill_mode='nearest'
)

data_gen = image.ImageDataGenerator(
    # define the preprocessing function that should be applied to all images
    preprocessing_function=preprocess_input,
    # fill_mode='nearest',
    # rotation_range=20,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # horizontal_flip=True, 
    # zoom_range=0.2,
    # shear_range=0.2
)

In [ ]:
from keras.models import load_model

# Define the directory containing your saved model
model_dir = "../models/"

# Specify the filename of your saved model
model_filename = "efficient_unfrozen_12.h5"

# Load the saved model
loaded_model = load_model(os.path.join(model_dir, model_filename))

loaded_model.summary()

In [ ]:
img_augmentation_layers = [
    layers.RandomRotation(factor=0.15),
    layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
    layers.RandomFlip(),
    layers.RandomContrast(factor=0.1),
]

def img_augmentation(images):
    for layer in img_augmentation_layers:
        images = layer(images)
    return images

In [ ]:
BATCH_SIZE = 32
TARGET_SIZE = (224,224,3)

train_datagen = ImageDataGenerator(
    rescale=1./255, featurewise_center=True, 
    featurewise_std_normalization=True, 
    rotation_range=20, width_shift_range=0.2,
    height_shift_range=0.2, horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./224)

train_generator = train_datagen.flow_from_directory(
    '../data/train',
    target_size=TARGET_SIZE[:2],
    batch_size=BATCH_SIZE,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    '../data/test',
    target_size=TARGET_SIZE[:2],
    batch_size=BATCH_SIZE,
    class_mode='categorical')

In [ ]:
hist_aug = loaded_model.fit(train_generator, steps_per_epoch=int(2400/32), epochs=10, 
          validation_data=validation_generator)